# Urinary system diseases classification
#### Presumptive classification of nephritis from inflammation

This notebook aims to create a classification model for the dataset described in here https://www.kaggle.com/yamqwe/acute-inflammationse


```
Credit:
Jacek Czerniak, Ph.D., Assistant ProfessorSystems Research InstitutePolish Academy of SciencesLaboratory of Intelligent Systems ul. Newelska 6, Room 21801-447 Warszawa, Polande-mail:
jacek.czerniak '@' ibspan.waw.pl or jczerniak '@' ukw.edu.pl
```

In [39]:
import pandas as pd

In [40]:
data_file = 'data/diagnosis.data'
columns = ['Temperature of patient', 'Occurrence of nausea', 'Lumbar pain', 'Urine pushing (continuous need for urination)', 'Micturition pains', 'Burning of urethra, itch, swelling of urethra outlet', 'Inflammation of urinary bladder', 'Nephritis of renal pelvis origin']
data_df = pd.read_csv(filepath_or_buffer=data_file, sep='\t', header=None, names=columns, encoding='utf-16')
data_df.head()

,Temperature of patient,Occurrence of nausea,Lumbar pain,Urine pushing (continuous need for urination),Micturition pains,"Burning of urethra, itch, swelling of urethra outlet",Inflammation of urinary bladder,Nephritis of renal pelvis origin
0,"35,5",no,yes,no,no,no,no,no
1,"35,9",no,no,yes,yes,yes,yes,no
2,"35,9",no,yes,no,no,no,no,no
3,"36,0",no,no,yes,yes,yes,yes,no
4,"36,0",no,yes,no,no,no,no,no


## Feature engineering

### Drop missing values

In [41]:
data_df.shape

(120, 8)

In [42]:
data_df = data_df.dropna(axis=0)
data_df.shape

(120, 8)

There are no missing values

### Convert text to numbers

In [43]:
for c in columns[1:]:
    data_df.loc[data_df[c]=='no',c] = 0
    data_df.loc[data_df[c]=='yes',c] = 1
data_df.head()


,Temperature of patient,Occurrence of nausea,Lumbar pain,Urine pushing (continuous need for urination),Micturition pains,"Burning of urethra, itch, swelling of urethra outlet",Inflammation of urinary bladder,Nephritis of renal pelvis origin
0,"35,5",0,1,0,0,0,0,0
1,"35,9",0,0,1,1,1,1,0
2,"35,9",0,1,0,0,0,0,0
3,"36,0",0,0,1,1,1,1,0
4,"36,0",0,1,0,0,0,0,0


### Create labels  

- Check if one patient can have both diseases

In [44]:
data_df[(data_df['Inflammation of urinary bladder'] == 1) & (data_df['Nephritis of renal pelvis origin'] == 1)].shape

(19, 8)

- Check if one patient don't have any disease

In [45]:
data_df[(data_df['Inflammation of urinary bladder'] == 0) & (data_df['Nephritis of renal pelvis origin'] == 0)].shape

(30, 8)

Create a Diagnosis column which will serve as our label. It can have values of
- 0 : if patient has no disease
- 1 : if patient was diagnosed with just "Nephritis of renal pelvis origin"
- 2 : if patient was diagnosed with just "Inflammation of urinary bladder"
- 3 : if patient was diagnosed with both diseases

In [46]:
import numpy as np

In [47]:
conditions = [
    ((data_df['Inflammation of urinary bladder'] ==
     0) & (data_df['Nephritis of renal pelvis origin'] == 0)),
    ((data_df['Inflammation of urinary bladder'] ==
     0) & (data_df['Nephritis of renal pelvis origin'] == 1)),
    ((data_df['Inflammation of urinary bladder'] ==
     1) & (data_df['Nephritis of renal pelvis origin'] == 0)),
    ((data_df['Inflammation of urinary bladder'] ==
     1) & (data_df['Nephritis of renal pelvis origin'] == 1))
]
values = [0, 1, 2, 3]
data_df['Diagnosis'] = np.select(conditions, values)
data_df.head()


,Temperature of patient,Occurrence of nausea,Lumbar pain,Urine pushing (continuous need for urination),Micturition pains,"Burning of urethra, itch, swelling of urethra outlet",Inflammation of urinary bladder,Nephritis of renal pelvis origin,Diagnosis
0,"35,5",0,1,0,0,0,0,0,0
1,"35,9",0,0,1,1,1,1,0,2
2,"35,9",0,1,0,0,0,0,0,0
3,"36,0",0,0,1,1,1,1,0,2
4,"36,0",0,1,0,0,0,0,0,0


### Drop previous diagnosis columns

In [48]:
data_df = data_df.drop(['Inflammation of urinary bladder', 'Nephritis of renal pelvis origin'], axis=1)
data_df.head()

,Temperature of patient,Occurrence of nausea,Lumbar pain,Urine pushing (continuous need for urination),Micturition pains,"Burning of urethra, itch, swelling of urethra outlet",Diagnosis
0,"35,5",0,1,0,0,0,0
1,"35,9",0,0,1,1,1,2
2,"35,9",0,1,0,0,0,0
3,"36,0",0,0,1,1,1,2
4,"36,0",0,1,0,0,0,0


### Scale temperature

Convert temperature decimal character

In [49]:
data_df['Temperature of patient'] = data_df['Temperature of patient'].str.replace(',','.')
data_df.head()

,Temperature of patient,Occurrence of nausea,Lumbar pain,Urine pushing (continuous need for urination),Micturition pains,"Burning of urethra, itch, swelling of urethra outlet",Diagnosis
0,35.5,0,1,0,0,0,0
1,35.9,0,0,1,1,1,2
2,35.9,0,1,0,0,0,0
3,36.0,0,0,1,1,1,2
4,36.0,0,1,0,0,0,0


In [50]:
data_df = data_df.astype({'Temperature of patient':'float',
        'Occurrence of nausea':'int32',
        'Lumbar pain':'int32',
        'Urine pushing (continuous need for urination)':'int32',
        'Micturition pains':'int32',
        'Burning of urethra, itch, swelling of urethra outlet':'int32',
        'Diagnosis':'category',
})
data_df.dtypes

Temperature of patient                                   float64
Occurrence of nausea                                       int32
Lumbar pain                                                int32
Urine pushing (continuous need for urination)              int32
Micturition pains                                          int32
Burning of urethra, itch, swelling of urethra outlet       int32
Diagnosis                                               category
dtype: object

In [51]:
data_df: pd.DataFrame
data_df.describe(include='all')

,Temperature of patient,Occurrence of nausea,Lumbar pain,Urine pushing (continuous need for urination),Micturition pains,"Burning of urethra, itch, swelling of urethra outlet",Diagnosis
count,120.000000,120.000000,120.000000,120.000000,120.000000,120.000000,120.0
unique,NaN,NaN,NaN,NaN,NaN,NaN,4.0
top,NaN,NaN,NaN,NaN,NaN,NaN,2.0
freq,NaN,NaN,NaN,NaN,NaN,NaN,40.0
mean,38.724167,0.241667,0.583333,0.666667,0.491667,0.416667,NaN
std,1.819132,0.429888,0.495074,0.473381,0.502027,0.495074,NaN
min,35.500000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,37.100000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
50%,37.950000,0.000000,1.000000,1.000000,0.000000,0.000000,NaN
75%,40.600000,0.000000,1.000000,1.000000,1.000000,1.000000,NaN


In [52]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
data_df[['Temperature of patient']]=pd.DataFrame(scaler.fit_transform(data_df[['Temperature of patient']].astype(float)))
data_df.head()


,Temperature of patient,Occurrence of nausea,Lumbar pain,Urine pushing (continuous need for urination),Micturition pains,"Burning of urethra, itch, swelling of urethra outlet",Diagnosis
0,0.000000,0,1,0,0,0,0
1,0.066667,0,0,1,1,1,2
2,0.066667,0,1,0,0,0,0
3,0.083333,0,0,1,1,1,2
4,0.083333,0,1,0,0,0,0


In [65]:
from pickle import dump
import os
os.makedirs('model',exist_ok=True)
dump(scaler, open('model/scaler.pkl', 'wb'))